# 🤖 Predicting Human Preferences in Chatbot Arena | LLM Classification Finetuning 🧠

<h1 style="font-family: 'poppins'; font-weight: bold; color: Green;">👨‍💻Author: Dr. Muneeb Hassan</h1>

[![GitHub](https://img.shields.io/badge/GitHub-Profile-blue?style=for-the-badge&logo=github)](https://github.com/MUNEEB-HASSAN)  
[![Kaggle](https://img.shields.io/badge/Kaggle-Notebook-blue?style=for-the-badge&logo=kaggle)](https://www.kaggle.com/code/muneebhassansipra)  
[![LinkedIn](https://img.shields.io/badge/LinkedIn-Profile-blue?style=for-the-badge&logo=linkedin)](https://www.linkedin.com/in/muneebhassansipra)

---

### 📘 Competition Overview
This competition challenges us to predict which LLM response users will prefer during a head-to-head chatbot battle in the **Chatbot Arena**.

You'll work with real-world dialogue data and fine-tune models using **Reinforcement Learning from Human Feedback (RLHF)** concepts — an essential skill in modern AI alignment.

---

### 🎯 Goal
Predict the preferred response (`model_a`, `model_b`, or `tie`) based on the conversation **prompt** and the two **LLM-generated responses**.

> 📌 Evaluation Metric: **Log Loss** on multi-class probabilities:  
> `winner_model_a`, `winner_model_b`, `winner_tie`

---

### 🛠️ Solution Approach
- Text Embedding via `KerasNLP` / `SentenceTransformer`
- Feature engineering from prompt and responses
- Deep Learning classifier (Dense NN)
- Submission ready with log-loss optimized output

Let's get started!


# Importing libraries

In [ ]:
# Suppress TensorFlow and CUDA warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress TensorFlow INFO and WARNING logs

# Optional: disable GPU if not needed
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Core Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# NLP & Modeling
import tensorflow as tf
import keras
import keras_nlp
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from tqdm import tqdm

# Miscellaneous
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_colwidth', None)

# Confirm library versions
print("✅ TensorFlow version:", tf.__version__)
print("✅ KerasNLP version:", keras_nlp.__version__)


# Load Dataset

In [ ]:
data_path = "/kaggle/input/llm-classification-finetuning/"

train_df = pd.read_csv(data_path + "train.csv")
test_df = pd.read_csv(data_path + "test.csv")
sample_submission = pd.read_csv(data_path + "sample_submission.csv")

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)
print("Sample submission shape:", sample_submission.shape)

train_df.head()


# EDA and Data Wrangling

In [ ]:
train_df.describe()

In [ ]:
# Basic info
train_df.info()

# Check for missing values
print("\n🧼 Missing values:\n", train_df.isnull().sum())

# Check target distribution (multi-label one-hot encoded)
print("\n🎯 Winner Distribution:")
target_counts = train_df[["winner_model_a", "winner_model_b", "winner_tie"]].sum()
print(target_counts)

# Plot target distribution
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")
plt.figure(figsize=(6,4))
sns.barplot(x=target_counts.index, y=target_counts.values, palette="viridis")
plt.title("Distribution of Winner Classes")
plt.ylabel("Count")
plt.xlabel("Winner")
plt.show()


In [ ]:
# Combine prompt and responses into single strings for model input
train_df["text_a"] = train_df["prompt"] + " " + train_df["response_a"]
train_df["text_b"] = train_df["prompt"] + " " + train_df["response_b"]
test_df["text_a"] = test_df["prompt"] + " " + test_df["response_a"]
test_df["text_b"] = test_df["prompt"] + " " + test_df["response_b"]


In [ ]:
# Convert one-hot targets to single integer labels
def get_winner(row):
    if row["winner_model_a"] == 1:
        return 0
    elif row["winner_model_b"] == 1:
        return 1
    else:
        return 2

train_df["target"] = train_df.apply(get_winner, axis=1)


In [ ]:
train_df[["winner_model_a", "winner_model_b", "winner_tie", "target"]].head()


In [ ]:
train_df[['winner_model_a', 'winner_model_b', 'winner_tie']].sum()


In [ ]:
sample = train_df.sample(1).iloc[0]
print("Prompt:\n", sample['prompt'])
print("\nResponse A:\n", sample['response_a'])
print("\nResponse B:\n", sample['response_b'])
print("\nWinner:", ["model_a", "model_b", "tie"][sample['target']])


In [ ]:
# Length-based features
train_df["prompt_len"] = train_df["prompt"].apply(lambda x: len(x.split()))
train_df["response_a_len"] = train_df["response_a"].apply(lambda x: len(x.split()))
train_df["response_b_len"] = train_df["response_b"].apply(lambda x: len(x.split()))


In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(train_df["prompt_len"], bins=50, kde=True, color="orange", label="Prompt")
sns.histplot(train_df["response_a_len"], bins=50, kde=True, color="blue", label="Response A", alpha=0.5)
sns.histplot(train_df["response_b_len"], bins=50, kde=True, color="green", label="Response B", alpha=0.5)
plt.legend()
plt.title("Length Distribution (Token Count)")
plt.xlabel("Token Count")
plt.show()


In [ ]:
# Create winning response length
train_df["winner_len"] = train_df.apply(
    lambda row: len(row["response_a"].split()) if row["target"] == 0 
    else len(row["response_b"].split()) if row["target"] == 1 
    else (len(row["response_a"].split()) + len(row["response_b"].split())) / 2,
    axis=1
)

# Compare winner vs average length
plt.figure(figsize=(6, 4))
sns.boxplot(data=train_df, x="target", y="winner_len", palette="pastel")
plt.title("Winning Response Lengths by Target Class")
plt.xlabel("Winner (0: A, 1: B, 2: Tie)")
plt.ylabel("Token Length")
plt.show()


In [ ]:
import re

def clean_text(text):
    text = re.sub(r'\s+', ' ', text)          # Remove extra spaces
    text = text.replace('\n', ' ')            # Remove line breaks
    text = re.sub(r"[^a-zA-Z0-9.,;!?()\[\]{}'\":/\-–—\s]", "", text)  # Remove weird chars
    return text.strip()

for col in ["prompt", "response_a", "response_b"]:
    train_df[col] = train_df[col].astype(str).apply(clean_text)
    test_df[col] = test_df[col].astype(str).apply(clean_text)


In [ ]:
# Duplicate prompts
print("🔁 Duplicate prompts:", train_df["prompt"].duplicated().sum())

# Any samples where response_a and response_b are the same?
train_df["is_same_response"] = train_df["response_a"] == train_df["response_b"]
print("🟰 Identical responses in train:", train_df["is_same_response"].sum())


In [ ]:
prompt_counts = train_df["prompt"].value_counts().to_dict()
train_df["prompt_freq"] = train_df["prompt"].map(prompt_counts)


In [ ]:
# Check the label distribution for identical responses
identical_rows = train_df[train_df["is_same_response"] == True]
print(identical_rows["target"].value_counts())


In [ ]:
# OPTION 1 — Drop them (recommended)
train_df = train_df[train_df["is_same_response"] == False]

# OPTION 2 — Set winner to tie (less preferred)
# train_df.loc[train_df["is_same_response"] == True, "target"] = 2


# Cosine similarity

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Create pairwise TF-IDF vectors
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)
response_pairs = train_df["response_a"] + " " + train_df["response_b"]
tfidf_matrix = tfidf.fit_transform(response_pairs)

# Compute cosine similarity between response_a and response_b
response_a_vecs = tfidf.transform(train_df["response_a"])
response_b_vecs = tfidf.transform(train_df["response_b"])

train_df["cosine_sim"] = [
    cosine_similarity(response_a_vecs[i], response_b_vecs[i])[0][0]
    for i in range(len(train_df))
]


# Jaccard Similarity

In [ ]:
def jaccard_similarity(str1, str2):
    a = set(str1.lower().split())
    b = set(str2.lower().split())
    return len(a & b) / len(a | b)

train_df["jaccard_sim"] = train_df.apply(
    lambda row: jaccard_similarity(row["response_a"], row["response_b"]), axis=1
)


# levenshtein 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Only plot cosine and jaccard similarity
for col in ["cosine_sim", "jaccard_sim"]:
    plt.figure(figsize=(8, 4))
    sns.boxplot(x=train_df["target"], y=train_df[col])
    plt.title(f"{col} by Target (0=A win, 1=B win, 2=Tie)")
    plt.xlabel("Target")
    plt.ylabel("Similarity")
    plt.show()


In [ ]:
print(train_df[["cosine_sim", "jaccard_sim", "target"]].corr())


# Token overlap

In [ ]:
def token_overlap(a, b):
    set_a = set(str(a).lower().split())
    set_b = set(str(b).lower().split())
    return len(set_a & set_b)

train_df["token_overlap_ab"] = train_df.apply(lambda row: token_overlap(row["response_a"], row["response_b"]), axis=1)
train_df["token_overlap_ap"] = train_df.apply(lambda row: token_overlap(row["response_a"], row["prompt"]), axis=1)
train_df["token_overlap_bp"] = train_df.apply(lambda row: token_overlap(row["response_b"], row["prompt"]), axis=1)


In [ ]:
def normalized_token_overlap(a, b):
    set_a = set(str(a).lower().split())
    set_b = set(str(b).lower().split())
    return len(set_a & set_b) / max(1, len(set_a | set_b))

train_df["norm_overlap_ab"] = train_df.apply(lambda row: normalized_token_overlap(row["response_a"], row["response_b"]), axis=1)


In [ ]:
features = [
    "cosine_sim",
    "jaccard_sim",
    "token_overlap_ab",
    "token_overlap_ap",
    "token_overlap_bp",
    # Add more features here if you've created them
]
X = train_df[features]
y = train_df["target"]


# Boosting Model comprison

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score

models = {
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42),
    "CatBoost": CatBoostClassifier(verbose=0, random_state=42),
    "LightGBM": LGBMClassifier(random_state=42)
}

for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    acc = accuracy_score(y_valid, preds)
    print(f"{name} Accuracy: {acc:.4f}")


In [ ]:
print(test_df.columns.tolist())


In [ ]:
import pandas as pd
from lightgbm import LGBMClassifier

# ✅ Recreate feature columns for both train and test
train_df["prompt_len"] = train_df["prompt"].str.len()
train_df["response_a_len"] = train_df["response_a"].str.len()
train_df["response_b_len"] = train_df["response_b"].str.len()

test_df["prompt_len"] = test_df["prompt"].str.len()
test_df["response_a_len"] = test_df["response_a"].str.len()
test_df["response_b_len"] = test_df["response_b"].str.len()

# ✅ Select feature columns
features = ["prompt_len", "response_a_len", "response_b_len"]

# ✅ Train model
X_train = train_df[features]
y_train = train_df["target"]
lgb_model = LGBMClassifier()
lgb_model.fit(X_train, y_train)

# ✅ Predict
X_test = test_df[features]
y_test_preds = lgb_model.predict(X_test)

# ✅ Submission
submission = pd.read_csv("/kaggle/input/llm-classification-finetuning/sample_submission.csv")
submission["target"] = y_test_preds
submission.to_csv("submission.csv", index=False)

print("✅ submission.csv saved and ready to submit.")


In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from lightgbm import LGBMClassifier
import numpy as np

# ✅ Stratified K-Fold setup
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
accuracies = []

# ✅ Loop over each fold
for fold, (train_idx, val_idx) in enumerate(skf.split(X_train, y_train)):
    print(f"\n📂 Fold {fold + 1}")
    
    X_tr, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]
    
    model = LGBMClassifier()
    model.fit(X_tr, y_tr)
    
    y_pred = model.predict(X_val_fold)
    acc = accuracy_score(y_val_fold, y_pred)
    print(f"✅ Accuracy: {acc:.4f}")
    accuracies.append(acc)

print(f"\n🎯 Mean Accuracy across folds: {np.mean(accuracies):.4f}")


In [ ]:
# ✅ Final model on full training data (using same 3 best features)
final_model = LGBMClassifier()
final_model.fit(X_train, y_train)

# ✅ Predict on test set
y_test_preds = final_model.predict(X_test)

# ✅ Map predictions to class labels if needed (0, 1, 2 to column names)
submission = pd.DataFrame({
    "id": test_df["id"],
    "prediction": y_test_preds
})

# ✅ Save to CSV
submission.to_csv("/kaggle/working/submission.csv", index=False)
print("📁 submission.csv saved!")


In [ ]:
# 🔁 Map class indices (0/1/2) to actual labels
label_map = {0: "model_a", 1: "model_b", 2: "tie"}
y_test_labels = [label_map[i] for i in y_test_preds]

# ✅ Prepare submission DataFrame
submission = pd.DataFrame({
    "id": test_df["id"],
    "prediction": y_test_labels
})

# ✅ Save to correct CSV
submission.to_csv("/kaggle/working/submission.csv", index=False)
print("📁 Correct submission.csv saved with", len(submission), "rows")


# Keras NLP model

# Combine prompt, response_a and response_b into a single string
train_df["text"] = train_df["prompt"] + " [SEP] " + train_df["response_a"] + " [SEP] " + train_df["response_b"]
test_df["text"] = test_df["prompt"] + " [SEP] " + test_df["response_a"] + " [SEP] " + test_df["response_b"]


import keras_nlp
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Load tokenizer and backbone
pretrained_model_name = "bert_base_en"
preprocessor = keras_nlp.models.BertPreprocessor.from_preset(pretrained_model_name)
backbone = keras_nlp.models.BertBackbone.from_preset(pretrained_model_name)


# Combine text
train_df["text"] = train_df["prompt"] + " [SEP] " + train_df["response_a"] + " [SEP] " + train_df["response_b"]
test_df["text"] = test_df["prompt"] + " [SEP] " + test_df["response_a"] + " [SEP] " + test_df["response_b"]

# Tokenize inputs using preprocessor
train_tokenized = preprocessor(tf.constant(train_df["text"].tolist()))
test_tokenized = preprocessor(tf.constant(test_df["text"].tolist()))


# Convert target
train_df["target"] = train_df[["winner_model_a", "winner_model_b", "winner_tie"]].idxmax(axis=1).map({
    "winner_model_a": 0,
    "winner_model_b": 1,
    "winner_tie": 2
})
y = to_categorical(train_df["target"], num_classes=3)

# Split on token_ids
X_train, X_val, y_train, y_val = train_test_split(
    train_tokenized["token_ids"].numpy(), y, test_size=0.2, random_state=42
)


# BERT 
**BERT (Bidirectional Encoder Representations from Transformers)** is a deep learning model developed by Google that understands the context of words in a sentence by looking at both the left and right sides (bidirectionally).
It uses a transformer architecture and is pre-trained on massive text data using tasks like masked language modeling.
BERT can be fine-tuned for various NLP tasks such as classification, question answering, and sentiment analysis.


# Define 3 inputs as required by BERT
input_ids = tf.keras.Input(shape=(None,), dtype=tf.int32, name="token_ids")
segment_ids = tf.keras.Input(shape=(None,), dtype=tf.int32, name="segment_ids")
padding_mask = tf.keras.Input(shape=(None,), dtype=tf.int32, name="padding_mask")

# Get BERT outputs
bert_outputs = backbone({
    "token_ids": input_ids,
    "segment_ids": segment_ids,
    "padding_mask": padding_mask
})

# Use pooled_output for classification
x = tf.keras.layers.Dropout(0.2)(bert_outputs["pooled_output"])
output = tf.keras.layers.Dense(3, activation="softmax")(x)

# Define model
model = tf.keras.Model(
    inputs={"token_ids": input_ids, "segment_ids": segment_ids, "padding_mask": padding_mask},
    outputs=output
)

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()


from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# ✅ One-hot encode target column
y = train_df["target"].values
y_cat = to_categorical(y, num_classes=3)

# ✅ Extract individual arrays
token_ids = train_tokenized["token_ids"].numpy()
segment_ids = train_tokenized["segment_ids"].numpy()
padding_mask = train_tokenized["padding_mask"].numpy()

# ✅ Print shapes
print("token_ids shape:", token_ids.shape)
print("segment_ids shape:", segment_ids.shape)
print("padding_mask shape:", padding_mask.shape)
print("y_cat shape:", y_cat.shape)  # Should be (57199, 3)

# ✅ Train/test split
X_token_train, X_token_val, y_train, y_val = train_test_split(
    token_ids, y_cat, test_size=0.2, random_state=42
)
X_seg_train, X_seg_val = train_test_split(segment_ids, test_size=0.2, random_state=42)
X_pad_train, X_pad_val = train_test_split(padding_mask, test_size=0.2, random_state=42)

# ✅ Assemble input dictionaries
X_train = {
    "token_ids": X_token_train,
    "segment_ids": X_seg_train,
    "padding_mask": X_pad_train
}

X_val = {
    "token_ids": X_token_val,
    "segment_ids": X_seg_val,
    "padding_mask": X_pad_val
}


import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import keras_nlp

# ✅ Reload data
train_df = pd.read_csv("/kaggle/input/llm-classification-finetuning/train.csv")

# ✅ Target column from winner flags
train_df["target"] = train_df[["winner_model_a", "winner_model_b", "winner_tie"]].idxmax(axis=1)
train_df["target"] = train_df["target"].map({"winner_model_a": 0, "winner_model_b": 1, "winner_tie": 2})

# ✅ Join prompt and responses
texts = train_df["prompt"] + " [SEP] " + train_df["response_a"] + " [SEP] " + train_df["response_b"]
y_cat = to_categorical(train_df["target"].values, num_classes=3)

# ✅ Use working model: bert_base_en_uncased
preprocessor = keras_nlp.models.BertPreprocessor.from_preset("bert_base_en_uncased", sequence_length=256)
backbone = keras_nlp.models.BertBackbone.from_preset("bert_base_en_uncased")
tokens = preprocessor(texts.tolist())

# ✅ Prepare inputs and split
X = {k: tokens[k].numpy() for k in ["token_ids", "segment_ids", "padding_mask"]}
X_train = {k: v[:45000] for k, v in X.items()}
X_val = {k: v[45000:] for k, v in X.items()}
y_train = y_cat[:45000]
y_val = y_cat[45000:]

# ✅ Build model
i1 = tf.keras.Input((256,), dtype=tf.int32, name="token_ids")
i2 = tf.keras.Input((256,), dtype=tf.int32, name="segment_ids")
i3 = tf.keras.Input((256,), dtype=tf.int32, name="padding_mask")
x = backbone({"token_ids": i1, "segment_ids": i2, "padding_mask": i3})["pooled_output"]
x = tf.keras.layers.Dropout(0.3)(x)
out = tf.keras.layers.Dense(3, activation="softmax")(x)
model = tf.keras.Model(inputs=[i1, i2, i3], outputs=out)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# ✅ Train (small batch for memory)
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=2, batch_size=8)


# kerasnlp takes lots of time and dont give best accurcy thats why i have to shift next model of transformer

# roberta-base model of transformer

from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)


from transformers import create_optimizer

# ✅ Set training details
batch_size = 16
num_epochs = 3

# 🔁 Set number of training steps
steps_per_epoch = len(y_train) // batch_size

# ✅ Create HuggingFace-compatible optimizer
optimizer, _ = create_optimizer(
    init_lr=2e-5,
    num_train_steps=steps_per_epoch * num_epochs,
    num_warmup_steps=0
)

# ✅ Compile the model
model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)


texts = train_df["prompt"] + " " + train_df["response_a"] + " " + train_df["response_b"]


from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# ✅ Step 1: Create input texts
texts = train_df["prompt"] + " " + train_df["response_a"] + " " + train_df["response_b"]

# ✅ Step 2: Target labels (0=model_a, 1=model_b, 2=tie)
y = train_df["target"].values
y_cat = to_categorical(y, num_classes=3)

# ✅ Step 3: Tokenize using HuggingFace
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
tokens = tokenizer(
    texts.tolist(),
    padding="max_length",
    truncation=True,
    max_length=256,
    return_tensors="np"  # ✅ convert directly to NumPy for sklearn compatibility
)

# ✅ Step 4: Train/Val Split
X_token_train, X_token_val, X_pad_train, X_pad_val, y_train, y_val = train_test_split(
    tokens["input_ids"], tokens["attention_mask"], y_cat,
    test_size=0.2, random_state=42
)

# ✅ Step 5: Format for HuggingFace TF models
X_train = {
    "input_ids": X_token_train,
    "attention_mask": X_pad_train
}
X_val = {
    "input_ids": X_token_val,
    "attention_mask": X_pad_val
}


from transformers import TFRobertaForSequenceClassification
from transformers import create_optimizer
import tensorflow as tf

# ✅ Step 1: Load model (Roberta-base for 3-class classification)
model = TFRobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=3)

# ✅ Step 2: Create optimizer with learning rate warmup
batch_size = 16
num_epochs = 2
steps_per_epoch = len(X_train["input_ids"]) // batch_size
total_train_steps = steps_per_epoch * num_epochs

optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_train_steps=total_train_steps,
    num_warmup_steps=0
)

# ✅ Step 3: Compile the model
model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

# ✅ Step 4: Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=num_epochs,
    batch_size=batch_size
)


# resulsts
 i run boosting models and compare but lightGBM give best results then kerasNLP and TFReborta these models takes lots of time but low accurcy also have overfitnig problems if pc is best increase epoch,reuglrization and early stoping techniques to avoid overfit,